# EE364 Coin Flipping Example
This is a simple example for a random experiment that we will perform as a class in our first meeting.  It illustrates a number of topics in the class which we will discuss during the semester, including the difference between probability ans statistics.  We can return to this as we progress during the semester.

In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

This simulates a class of size `N_students` where each student flips a coin (with heads probability `p`) `N_flips` times.  

In [2]:
N_students = 100000 # <<---- change this to adjust class size
N_flips = 10
p = 1.0 / 2.0  
np.random.seed(123)  # <<---- change this to get a "new class"

This is a well-known problem in probability, so the results for the theoretical model for this experiment are built into Python.  The following code snippet fills produces an array `binomial_pmf` with `binomial_pmf[k]` being the probability that `k` heads will be observed when a coin is flipped `N_flips` times.

In [3]:
heads = np.arange(0, N_flips + 1)
binomial_pmf = stats.binom.pmf(heads, N_flips, p)

Python also has built-in functions to run this experiment `N_students` times -- note that this is different from the theory above since it is simulating the actual random experiments of flipping the coin.

In [8]:
flip_results = np.random.binomial(N_flips, p, N_students)
flip_counts = np.zeros(N_flips + 1)

## count the number of times k flips were obtained in these N_students experiments
for k in range(N_flips + 1):
    flip_counts[k] = np.sum(flip_results == k)

This computes an estimate of `p`, the probability of heads, based on the experiments.  It also computes a "Margin of Error" for our estimate. 

In [ ]:
p_hat = flip_counts / N_students
MOE = np.zeros( (2, N_flips + 1) )
MOE[0] = 1.96 * np.sqrt( p_hat * (1.0 - p_hat) / N_students  )

print("number of students =",N_students)
print("number of flips =",N_flips)
print("probability of heads =",p)

print("\n\nk p(k) n(k) p_hat(k) MOE(k)\n")

for k in range(len(p_hat)):
    if flip_counts[k] == 0 : 
        MOE[1][k] = 3.0 / N_students
    else:
        MOE[1][k] = MOE[0][k]
    print(k,"{:0.4g}".format(binomial_pmf[k]), flip_counts[k], "{:0.4g}".format(p_hat[k]), "{:0.4g}".format(MOE[1][k]), sep=" ")

"""
print("\n\n")
for k in range(len(p_hat)):
    print(p_hat[k])
"""

MOE[0] = np.clip(MOE[0], 0, p_hat )  # limit confidence region to be > 0
